In [3]:
from datetime import datetime
from typing import Annotated, Any

import pytz
from dateutil.parser import parse

from pydantic import BaseModel, AfterValidator, BeforeValidator

def make_utc(dt: datetim) -> datetim:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt=dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value

DateTimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]
    



In [4]:
class Model(BaseModel):
    dt: DateTimeUTC

In [5]:
def dt_json_serializer(dt: datetime) -> str:
    return dt.strftime("%Y/%m/%d %I:%M %p UTC")

In [6]:
dt_json_serializer(datetime(2020, 1, 1, 12, 0, 0))

'2020/01/01 12:00 PM UTC'

In [7]:
from pydantic import field_serializer

In [8]:
class Model(BaseModel):
    dt: DateTimeUTC

    @field_serializer("dt", when_used="json-unless-none")
    def serialize_dt_to_json(self, value: datetime) -> str:
        return dt_json_serializer(value)

In [9]:
m = Model(dt="2020/1/1 3pm")
m

Model(dt=datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>))

In [10]:
m.model_dump()

{'dt': datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>)}

In [11]:
m.model_dump_json()

'{"dt":"2020/01/01 03:00 PM UTC"}'

In [12]:
from pydantic import PlainSerializer

DateTimeUTC = Annotated[
    datetime,
    BeforeValidator(parse_datetime),
    AfterValidator(make_utc),
    PlainSerializer(dt_json_serializer, when_used="json-unless-none")
]

In [13]:
class Model(BaseModel):
    dt: DateTimeUTC

In [14]:
m = Model(dt="2020/1/1 3pm")

In [15]:
m

Model(dt=datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>))

In [16]:
m.model_dump_json()

'{"dt":"2020/01/01 03:00 PM UTC"}'